# IMPORT

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [8, 4.5]
plt.rcParams['figure.dpi'] = 300


# 텐서 씨드 고정
import tensorflow as tf
import random
import os
#글로벌 시드
class config:
    seed = 1234
    device = "cuda:0"    

def seed_everything(seed: int = 1234):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    
seed_everything(config.seed)

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

seed =1234

pd.set_option('display.max_rows', 10, 'display.max_columns', 500, 'max_colwidth', None)

In [ ]:
break

In [2]:
# df = pd.read_csv('./kospi.csv')
df = pd.read_csv('./baseline_hv.csv')
col_2 = pd.read_csv("./data/cols_2.csv", index_col = 'Unnamed: 0') # TOP17 에서 17C2
col_3 = pd.read_csv("./data/cols_3.csv", index_col = 'Unnamed: 0') # TOP17 에서 17C3
col_4 = pd.read_csv("./data/cols_4.csv", index_col = 'Unnamed: 0') # TOP17 에서 17C4
df.shape

(4598, 55)

In [5]:
df.set_index('date', inplace=True)

In [9]:
# 전체 보조지표 
cols = ['HV','MA5', 'MA20',
       'MA60', 'MA120', 'MA5_gra', 'MA20_gra', 'MA60_gra', 'MA120_gra',
       'V_MA5', 'V_MA20', 'V_MA60', 'V_MA120', 'EMA5', 'EMA20', 'EMA60',
       'EMA120', 'MACD', 'MACD_sig9', 'MACD_OSC', 'Fast%K', 'Fast%D', 'Slow%K',
       'Slow%D', 'RSI14', 'RSI signal', 'CCI', 'A_Ratio', 'B_Ratio', 'ATR',
       'UpDI', 'DownDI', 'MFI', 'Momentum', 'Sonar', 'Sonar_sig5', 'STDEV',
       'Sigma', 'TRIX', 'VR', 'WILLR', 'Psy', 'Disparity20', 'Disparity60',
       'ADX', 'ADOSC', 'Volume_OSC', 'upperband', 'middleband', 'lowerband']

In [10]:
df = df[cols]

In [14]:
feature_df = df.loc['2003-09-29':'2021-09-28', :]
print(feature_df.shape)
feature_df.head()

(4532, 50)


,HV,MA5,MA20,MA60,MA120,MA5_gra,MA20_gra,MA60_gra,MA120_gra,V_MA5,V_MA20,V_MA60,V_MA120,EMA5,EMA20,EMA60,EMA120,MACD,MACD_sig9,MACD_OSC,Fast%K,Fast%D,Slow%K,Slow%D,RSI14,RSI signal,CCI,A_Ratio,B_Ratio,ATR,UpDI,DownDI,MFI,Momentum,Sonar,Sonar_sig5,STDEV,Sigma,TRIX,VR,WILLR,Psy,Disparity20,Disparity60,ADX,ADOSC,Volume_OSC,upperband,middleband,lowerband
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2003-09-29,0.000000,42.146221,42.842339,0.0,0.0,-0.003142,-0.000810,0.0,0.0,3.822771e+07,3.386586e+07,0.0,0.0,42.147814,42.477199,0.0,0.0,0.115163,0.362327,-0.247163,-372.336678,-397.842859,-442.619289,-470.122809,40.902448,46.689368,-113.005045,82.720811,-108.532695,15.995007,1.485662,2.240238,39.719839,-0.708041,-0.468851,-0.132596,0.612120,-1.225259,0.065066,83.958058,-472.336678,38.250,97.591753,0.0,18.762568,-3.909975e+09,0.484744,44.066579,42.842339,41.618099
2003-09-30,0.046147,41.847145,42.769250,0.0,0.0,-0.007730,-0.002238,0.0,0.0,4.313977e+07,3.470818e+07,0.0,0.0,41.951517,42.389744,0.0,0.0,0.033484,0.296558,-0.263074,-350.208431,-371.826241,-410.228375,-442.135894,41.405000,45.631824,-120.113868,72.439129,-108.628216,15.947273,1.391601,2.276410,40.754490,-1.214163,-0.717700,-0.327631,0.669568,-1.742624,0.029091,61.175908,-450.208431,28.250,96.506024,0.0,19.247360,-4.059619e+09,4.466314,44.108386,42.769250,41.430114
2003-10-01,0.111621,41.927376,42.740048,0.0,0.0,0.002179,-0.000942,0.0,0.0,4.484279e+07,3.536592e+07,0.0,0.0,42.159342,42.407387,0.0,0.0,0.050163,0.247279,-0.197116,-324.533998,-349.026369,-373.654201,-408.833955,47.815283,46.068737,-42.831090,83.965633,-108.920230,15.999634,1.783344,2.099806,44.213527,-0.000697,-0.808805,-0.488022,0.667278,-0.226977,0.004674,68.210780,-424.533998,37.875,99.353199,0.0,18.627031,-4.085364e+09,8.823151,44.074605,42.740048,41.405491
2003-10-02,0.113955,42.126832,42.717885,0.0,0.0,0.005389,-0.000766,0.0,0.0,4.280619e+07,3.591528e+07,0.0,0.0,42.374368,42.445199,0.0,0.0,0.080961,0.214015,-0.133055,-315.254086,-329.998838,-351.053361,-378.311979,49.982312,46.851770,4.260297,83.953248,-108.931287,16.000054,1.962065,1.947122,48.765714,-0.577864,-0.772456,-0.582833,0.660247,0.099223,-0.008533,70.947579,-415.254086,40.125,100.166147,0.0,17.712021,-4.208542e+09,8.880720,44.038379,42.717885,41.397392
2003-10-03,0.127419,42.477224,42.737588,0.0,0.0,0.009944,0.000424,0.0,0.0,4.357655e+07,3.650078e+07,0.0,0.0,42.711794,42.534861,0.0,0.0,0.150613,0.201335,-0.050722,-295.685126,-311.824403,-331.603664,-352.103742,55.713920,48.624775,76.695304,85.928471,-108.888583,16.064216,2.546638,1.794159,51.903948,0.114629,-0.492226,-0.552631,0.671952,0.955635,-0.007620,86.588038,-395.685126,49.500,101.891182,0.0,17.789960,-4.319473e+09,9.854309,44.081492,42.737588,41.393684


In [15]:
label_df = df.loc['2003-09-29':'2021-09-28', 'HV']
print(label_df.shape)
label_df.head()

(4532,)


date
2003-09-29    0.000000
2003-09-30    0.046147
2003-10-01    0.111621
2003-10-02    0.113955
2003-10-03    0.127419
Name: HV, dtype: float64

# 단일지표 선택

In [23]:
# 기간지정
train_start_date      = '2003-09-29'
train_end_date_1st    = '2018-09-28'
train_end_date_2st    = '2016-09-28'
test_start_date_2st   = '2018-09-29'
test_start_date_2st   = '2016-09-29'
test_end_date         = '2021-10-28'

# 전체 보조지표
cols = [
         'MA5', 'MA20',
       'MA60', 'MA120', 'MA5_gra', 'MA20_gra', 'MA60_gra', 'MA120_gra',
       'V_MA5', 'V_MA20', 'V_MA60', 'V_MA120', 'EMA5', 'EMA20', 'EMA60',
       'EMA120', 'MACD', 'MACD_sig9', 'MACD_OSC', 'Fast%K', 'Fast%D', 'Slow%K',
       'Slow%D', 'RSI14', 'RSI signal', 'CCI', 'A_Ratio', 'B_Ratio', 'ATR',
       'UpDI', 'DownDI', 'MFI', 'Momentum', 'Sonar', 'Sonar_sig5', 'STDEV',
       'Sigma', 'TRIX', 'VR', 'WILLR', 'Psy', 'Disparity20', 'Disparity60',
       'ADX', 'ADOSC', 'Volume_OSC', 'upperband', 'middleband', 'lowerband'
         ]

# 입력 파라미터 feature, label -> numpy type
def make_sequence_dataset(feature, label, window_size):
    feature_list = [] # 생성될 feature list
    label_list = []  # 생성될 label list
    
    for i in range(len(feature)-window_size):
        feature_list.append(feature[i:i+window_size])
        label_list.append(label[i+window_size])
    return np.array(feature_list), np.array(label_list) # 3차원으로 만들어줌

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
mm = MinMaxScaler()


rows = []
models = []
histories = []
for col in cols:
    row = []

    #X, y 분리
    feature_df = df[['HV', col]].loc['2003-09-29':'2021-09-28', :]
    label_df = df['HV'].loc['2003-09-29':'2021-09-28']
        
    #train_test_split
    X_train_2st = feature_df.loc[train_start_date:train_end_date_1st]
    y_train_1st = label_df.loc[train_start_date:train_end_date_1st]
    X_test_1st = feature_df.loc[test_start_date_2st:test_end_date]
    y_test_1st = label_df.loc[test_start_date_2st:test_end_date]

    # 스케일링
    mm.fit(X_train_2st)
    X_train_1st_sc = mm.transform(X_train_2st)
    X_test_1st_sc = mm.transform(X_test_1st)
    mm.fit(np.array(y_train_1st).reshape(-1,1))
    y_train_1st_sc = mm.transform(np.array(y_train_1st).reshape(-1,1))
    y_test_1st_sc = mm.transform(np.array(y_test_1st).reshape(-1,1))

    window_size = 20
    X_train_2st, y_train_1st = make_sequence_dataset(X_train_1st_sc, y_train_1st_sc, window_size)
    X_test_1st, y_test_1st = make_sequence_dataset(X_test_1st_sc, y_test_1st_sc, window_size)

    # 모델
    model_118 = Sequential()
    model_118.add(LSTM(units = 10, activation = 'relu', return_sequences = True, input_shape = X_train_2st[0].shape))
    model_118.add(Dense(units = 1))


    sgd = optimizers.SGD(learning_rate = 0.0001)
    early_stop = EarlyStopping(monitor = 'val_loss', patience = 5)
    history = model_118.fit(X_train_2st, y_train_1st, validation_data = (X_test_1st, y_test_1st), epochs=30, 
                batch_size=15, callbacks = [early_stop])
    val_loss = history.history['val_loss']
    rmse_score = np.sqrt(np.min(val_loss))

    models.append(model_118)
    histories.append(history)

    row.append(col)
    row.append(rmse_score)
    rows.append(row)

rmse_df = pd.DataFrame(rows, columns=[['col', 'RMSE'])

Epoch 1/30
251/251 [==============================] - 5s 12ms/step - loss: 0.0088 - val_loss: 0.0144
Epoch 2/30
251/251 [==============================] - 2s 9ms/step - loss: 0.0051 - val_loss: 0.0110
Epoch 3/30
251/251 [==============================] - 3s 11ms/step - loss: 0.0043 - val_loss: 0.0102
Epoch 4/30
251/251 [==============================] - 3s 11ms/step - loss: 0.0040 - val_loss: 0.0098
Epoch 5/30
251/251 [==============================] - 2s 9ms/step - loss: 0.0038 - val_loss: 0.0103
Epoch 6/30
251/251 [==============================] - 3s 10ms/step - loss: 0.0037 - val_loss: 0.0095
Epoch 7/30
251/251 [==============================] - 2s 9ms/step - loss: 0.0036 - val_loss: 0.0086
Epoch 8/30
251/251 [==============================] - 2s 9ms/step - loss: 0.0035 - val_loss: 0.0093
Epoch 9/30
251/251 [==============================] - 3s 12ms/step - loss: 0.0033 - val_loss: 0.0088
Epoch 10/30
251/251 [==============================] - 4s 16ms/step - loss: 0.0033 - val_loss: 

# 단일지표중 combination 선별

In [ ]:
train_start_date      = '2003-09-29'
train_end_date_1st    = '2018-09-28'
train_end_date_2st    = '2016-09-28'
test_start_date_1st   = '2018-09-29'
test_start_date_2st   = '2016-09-29'
test_end_date         = '2021-10-28'

cols = [
         'MA5', 'MA20',
       'MA60', 'MA120', 'MA5_gra', 'MA20_gra', 'MA60_gra', 'MA120_gra',
       'V_MA5', 'V_MA20', 'V_MA60', 'V_MA120', 'EMA5', 'EMA20', 'EMA60',
       'EMA120', 'MACD', 'MACD_sig9', 'MACD_OSC', 'Fast%K', 'Fast%D', 'Slow%K',
       'Slow%D', 'RSI14', 'RSI signal', 'CCI', 'A_Ratio', 'B_Ratio', 'ATR',
       'UpDI', 'DownDI', 'MFI', 'Momentum', 'Sonar', 'Sonar_sig5', 'STDEV',
       'Sigma', 'TRIX', 'VR', 'WILLR', 'Psy', 'Disparity20', 'Disparity60',
       'ADX', 'ADOSC', 'Volume_OSC', 'upperband', 'middleband', 'lowerband'
         ]

# 입력 파라미터 feature, label -> numpy type
def make_sequence_dataset(feature, label, window_size):
    feature_list = [] # 생성될 feature list
    label_list = []  # 생성될 label list
    
    for i in range(len(feature)-window_size):
        feature_list.append(feature[i:i+window_size])
        label_list.append(label[i+window_size])
    return np.array(feature_list), np.array(label_list) # 3차원으로 만들어줌

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
mm = MinMaxScaler()


rows = []
models = []
histories = []
# for i in range(col_3.shape[0]):
for i in range(col_3.shape[0]):   # 17 C 3 컬럼들
    row = []

    #X, y 분리
    feature_df = df[col_3.iloc[i, :].values].loc['2003-09-29':'2021-09-28', :]
    label_df = df['HV'].loc['2003-09-29':'2021-09-28']
        
    #train_test_split
    X_train_1st = feature_df.loc[train_start_date:train_end_date_1st]
    y_train_1st = label_df.loc[train_start_date:train_end_date_1st]
    X_test_1st = feature_df.loc[test_start_date_1st:test_end_date]
    y_test_1st = label_df.loc[test_start_date_1st:test_end_date]

    # MinMaxScaler
    mm.fit(X_train_1st)
    X_train_1st_sc = mm.transform(X_train_1st)
    X_test_1st_sc = mm.transform(X_test_1st)
    mm.fit(np.array(y_train_1st).reshape(-1,1))
    y_train_1st_sc = mm.transform(np.array(y_train_1st).reshape(-1,1))
    y_test_1st_sc = mm.transform(np.array(y_test_1st).reshape(-1,1))

    # window_size 조정
    window_size = 20
    X_train_1st, y_train_1st = make_sequence_dataset(X_train_1st_sc, y_train_1st_sc, window_size)
    X_test_1st, y_test_1st = make_sequence_dataset(X_test_1st_sc, y_test_1st_sc, window_size)

    # LSTM 모델생성
    model_118 = Sequential()
    model_118.add(LSTM(units = 10, activation = 'relu', return_sequences = True, input_shape = X_train_1st[0].shape))
    model_118.add(Dense(units = 1))
    sgd = optimizers.SGD(learning_rate = 0.0001)
    model_118.compile(loss = 'mse', optimizer = 'adam')

    early_stop = EarlyStopping(monitor = 'val_loss', patience = 5)
    
    history = model_118.fit(X_train_1st, y_train_1st, validation_data = (X_test_1st, y_test_1st), epochs=5, 
                batch_size=15, callbacks = [early_stop])
    
    val_loss = history.history['val_loss']
    rmse_score = np.sqrt(np.min(val_loss))

    models.append(model_118)
    histories.append(history)

    # 각각의 컬럼명 리스트안에 담기
    row.append(col_3.iloc[i, 0])
    row.append(col_3.iloc[i, 1])
    row.append(col_3.iloc[i, 2])
    # RMSE점수 담기
    row.append(rmse_score)
    rows.append(row)

rmse_df = pd.DataFrame(rows, columns=['col1', 'col2', 'col3', 'RMSE'])

# TO CSV

In [30]:
RMSE_1 = rmse_df.sort_values("RMSE")
RMSE_1.to_csv("./RMSE_1.csv", index=False)